# Student Details

***

**Name:** Hoai Nhan Nguyen <br>
**Student Number:** sba24098 <br>
**Course:** Higher Diploma in Science in Artificial Intelligence Applications 

***

# Assessment Task

Students are advised to review and adhere to the submission requirements documented after the assessment task.

# Assessment details

You are required to use the dataset attached “mye22final.xlsx”, that contains data on Estimates of the population for the UK, England, Wales, Scotland, and Northern Ireland for the years 2011 and 2022.

This Data is Licenced under open-government-licence v3.

https://www.nationalarchives.gov.uk/doc/open-government-licence/version/3/

Create an interactive Dashboard aimed at younger adults (18 - 35 years).

The dashboard may be created using ANY pythonic method within a Jupyter Notebook. PYTHON FILES May be used after they have been generated from the Jupyter Notebook or as helper files, but the main work MUST be completed in Jupyter Notebook. (You may (if you wish) Host the Dashboard using STREAMLIT or Similar). The Dashboard may be viewed in HTML format, within the notebook or as a standalone application based on the techniques covered in the class, (Tkinter, panel, dash, Jupyter-dash etc…). All final code and files MUST be uploaded to MOODLE and progress shown in your GitHub Classroom commits.

<b>YOU MAY NOT USE POWERBI, TABLEAU, LOOKER, or any other generalized tool for this assignment.</b>

The Dashboard must detail the following requirements:

* Be Interactive and include Hover functionality.
* Contain at least 2 Visualizations.
* Display population densities by Geographic location.
* Allow the user to select and display population densities by age AND by Gender.
* Display a comparison between the 2011 figures and the 2022 figures.

(70 Marks)

Discuss in detail your rationale and justification for all stages of data preparation for your visualizations and how your dashboard is designed with this demographic (younger adults (18 - 35 years)) in mind.  

(30 marks)
***

## Data Analysis and Preprocessing

**1. Data Understanding**

**Importing libraries**

In [2]:
import pandas as pd

**Loading Excel file and printing sheet names**

In [3]:
# Load the Excel file
xls = pd.ExcelFile("data/mye22final.xlsx")

# Print the names of all sheets
print(xls.sheet_names)

['Cover sheet', 'Contents', 'Notes', 'Geography guide', 'Related publications', 'MYE1', 'MYE2 - Persons', 'MYE2 - Females', 'MYE2 - Males', 'MYE4', 'MYE5', 'MYE6', 'MYEB1']


**Reviewing information on MYEB1 and MYE5 sheet in the Excel File**

In [4]:
df_uk_population = pd.read_excel('data/mye22final.xlsx', sheet_name='MYEB1')
df_uk_density = pd.read_excel('data/mye22final.xlsx', sheet_name='MYE5', skiprows=7)

In [5]:
# Check the head of the dataset
df_uk_population.head()

,code,name,geography,sex,age,population_2011,population_2022
0,K02000001,UNITED KINGDOM,Country,M,0,410357,355547
1,K02000001,UNITED KINGDOM,Country,M,1,403135,353474
2,K02000001,UNITED KINGDOM,Country,M,2,399930,366917
3,K02000001,UNITED KINGDOM,Country,M,3,404523,376045
4,K02000001,UNITED KINGDOM,Country,M,4,391667,380410


In [6]:
# Check the head of the dataset
df_uk_density.head()

,Code,Name,Geography,Area (sq km),Estimated Population mid-2022,2022 people per sq. km,Estimated Population mid-2011,2011 people per sq. km
0,K02000001,UNITED KINGDOM,Country,242494.5701,67596281,279,63285145,261
1,K03000001,GREAT BRITAIN,Country,228947.9162,65685738,287,61470827,268
2,K04000001,ENGLAND AND WALES,Country,151046.9855,60238038,399,56170927,372
3,E92000001,ENGLAND,Country,130310.4675,57106398,438,53107169,408
4,E12000001,NORTH EAST,Region,8580.7864,2683040,313,2596441,303


**Checking the data type of dataframe**

In [7]:
df_uk_population.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73528 entries, 0 to 73527
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   code             73528 non-null  object
 1   name             73528 non-null  object
 2   geography        73528 non-null  object
 3   sex              73528 non-null  object
 4   age              73528 non-null  int64 
 5   population_2011  73528 non-null  int64 
 6   population_2022  73528 non-null  int64 
dtypes: int64(3), object(4)
memory usage: 3.9+ MB


In [8]:
df_uk_density.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 404 entries, 0 to 403
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Code                           404 non-null    object 
 1   Name                           404 non-null    object 
 2   Geography                      404 non-null    object 
 3   Area (sq km)                   404 non-null    float64
 4   Estimated Population mid-2022  404 non-null    int64  
 5   2022 people per sq. km         404 non-null    int64  
 6   Estimated Population mid-2011  404 non-null    int64  
 7   2011 people per sq. km         404 non-null    int64  
dtypes: float64(1), int64(4), object(3)
memory usage: 25.4+ KB


**Adjusting the column names for df_uk_density dataframe**

In [9]:
df_uk_density.columns = [
    'code',
    'name',
    'geography',
    'area_sq_km',
    'population_2022',
    'person_per_sq_km_2022',
    'population_2011',
    'person_per_sq_km_2011'
]


**Checking any null or duplicate rows in dataframe.**

In [10]:
def checking_null_and_duplicate(df):
    if df.isnull().values.any():
        print("There is a null value in the DataFrame.")
    else:
        print("There is no null value in the DataFrame.")    
    
    # Check for duplicates
    if df.duplicated().sum() > 0:
        print("There is a duplicate value in the DataFrame.\n")
    else:
        print("There is no duplicate value in the DataFrame.\n")    

# Checking MYEB1 Sheet
print("== MYEB1 Sheet ==")
checking_null_and_duplicate(df_uk_population)

# Checking MYE5 Sheet
print("== MYE5 Sheet ==")
checking_null_and_duplicate(df_uk_density)



== MYEB1 Sheet ==
There is no null value in the DataFrame.
There is no duplicate value in the DataFrame.

== MYE5 Sheet ==
There is no null value in the DataFrame.
There is no duplicate value in the DataFrame.



**Checking the uniqueness of the dataframe**

In [11]:
df_uk_population.nunique()


code                404
name                404
geography            10
sex                   2
age                  91
population_2011    9462
population_2022    9743
dtype: int64

In [12]:
df_uk_density.nunique()

code                     404
name                     404
geography                 10
area_sq_km               404
population_2022          404
person_per_sq_km_2022    364
population_2011          404
person_per_sq_km_2011    359
dtype: int64

**Checking Unique values for the column "name".**

In [13]:
df_uk_population['name'].unique()

array(['UNITED KINGDOM', 'GREAT BRITAIN', 'ENGLAND AND WALES', 'ENGLAND',
       'NORTH EAST', 'County Durham', 'Darlington', 'Hartlepool',
       'Middlesbrough', 'Northumberland', 'Redcar and Cleveland',
       'Stockton-on-Tees', 'Tyne and Wear (Met County)', 'Gateshead',
       'Newcastle upon Tyne', 'North Tyneside', 'South Tyneside',
       'Sunderland', 'NORTH WEST', 'Blackburn with Darwen', 'Blackpool',
       'Cheshire East', 'Cheshire West and Chester', 'Cumberland',
       'Halton', 'Warrington', 'Westmorland and Furness',
       'Greater Manchester (Met County)', 'Bolton', 'Bury', 'Manchester',
       'Oldham', 'Rochdale', 'Salford', 'Stockport', 'Tameside',
       'Trafford', 'Wigan', 'Lancashire', 'Burnley', 'Chorley', 'Fylde',
       'Hyndburn', 'Lancaster', 'Pendle', 'Preston', 'Ribble Valley',
       'Rossendale', 'South Ribble', 'West Lancashire', 'Wyre',
       'Merseyside (Met County)', 'Knowsley', 'Liverpool', 'Sefton',
       'St. Helens', 'Wirral', 'YORKSHIRE AND

In [14]:
df_uk_density['name'].unique()

array(['UNITED KINGDOM', 'GREAT BRITAIN', 'ENGLAND AND WALES', 'ENGLAND',
       'NORTH EAST', 'County Durham', 'Darlington', 'Hartlepool',
       'Middlesbrough', 'Northumberland', 'Redcar and Cleveland',
       'Stockton-on-Tees', 'Tyne and Wear (Met County)', 'Gateshead',
       'Newcastle upon Tyne', 'North Tyneside', 'South Tyneside',
       'Sunderland', 'NORTH WEST', 'Blackburn with Darwen', 'Blackpool',
       'Cheshire East', 'Cheshire West and Chester', 'Cumberland',
       'Halton', 'Warrington', 'Westmorland and Furness',
       'Greater Manchester (Met County)', 'Bolton', 'Bury', 'Manchester',
       'Oldham', 'Rochdale', 'Salford', 'Stockport', 'Tameside',
       'Trafford', 'Wigan', 'Lancashire', 'Burnley', 'Chorley', 'Fylde',
       'Hyndburn', 'Lancaster', 'Pendle', 'Preston', 'Ribble Valley',
       'Rossendale', 'South Ribble', 'West Lancashire', 'Wyre',
       'Merseyside (Met County)', 'Knowsley', 'Liverpool', 'Sefton',
       'St. Helens', 'Wirral', 'YORKSHIRE AND

**Checking Unique values for the column "geography".**

In [15]:
df_uk_population['geography'].unique()

array(['Country', 'Region', 'Unitary Authority', 'Metropolitan County',
       'Metropolitan District', 'County', 'Non-metropolitan District',
       'London Borough', 'Council Area', 'Local Government District'],
      dtype=object)

In [16]:
df_uk_density['geography'].unique()

array(['Country', 'Region', 'Unitary Authority', 'Metropolitan County',
       'Metropolitan District', 'County', 'Non-metropolitan District',
       'London Borough', 'Council Area', 'Local Government District'],
      dtype=object)